In [1]:
import numpy as np
import pandas as pd
import cv2

In [3]:
reader = pd.read_csv('./data/driving_log.csv', usecols=['center','left','right','steering'] )

type(reader)

imgs = []
labels = []

cache = ["center","left","right"]
for index, row in reader.iterrows():
    # print(row['center'], row['steering'])
    
    file_path_center = './data/' + row['center']
    # file_path_left = './data/' + row['left']
    # file_path_right= './data/' + row['right']
    
    img_CENTER = cv2.imread(file_path_center)
    # img_LEFT = cv2.imread(file_path_left)
    # img_RIGHT = cv2.imread(file_path_right)
    steering_C = float(row['steering'])
    # steering_L= float(row['steering'])+0.02
    # steering_R = float(row['steering'])-0.02
    imgs.append(img_CENTER)
    labels.append(steering_C)
    # imgs.append(img_LEFT)
    # labels.append(steering_L)
    # imgs.append(img_RIGHT)
    # labels.append(steering_R)

augmented_images , augmented_measurements = [],[]

for image , mesure in zip(imgs,labels):
    augmented_images.append(image)
    augmented_measurements.append(mesure)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(mesure*-1.0)

In [20]:
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)


    
X_train[0].shape

(160, 320, 3)